# Weather preloader

Seems like current versions of Spine Toolbox and atlite have some issues when downloading full-year weather data on the fly, so I'll need to see if I can avoid these issues by preparing the necessary cutouts ahead of time.

In [ ]:
## Import necessary packages

import archetypebuildingweather as abw
import pandas as pd

In [ ]:
## Define weather preloader settings

countries = None # ["NL", "FR"]
weather_start = "2008-01-01"
weather_end = "2008-01-01"
timeslice = slice(weather_start, weather_end)
dask_kwargs = None #dict(scheduler="synchronous")
tmpdir = None #"data"
raster_path = "../AmBIENCe2ABM/data_sources/gfa_res_curr_density/data/gfa_res_curr_density_epsg4326.tif"


## Fetch desired shapefiles from mappings.

df_shapefiles = pd.read_csv("data_assumptions/shapefile_mappings.csv").set_index("country")
if countries is not None:
    df_shapefiles = df_shapefiles.loc[countries]
df_shapefiles

In [ ]:
## Preload weather data

cutouts = []
countries = df_shapefiles.index
for c in countries:
    print(c)
    shp = abw.Shapefile(df_shapefiles.loc[c]["shapefile_path"])
    cutout = abw.prepare_cutout(shp, weather_start, weather_end, dask_kwargs=dask_kwargs, tmpdir=tmpdir)
    cutouts.append(cutout)
    weights = {c: 1.0}
    raster, layout = abw.prepare_layout(shp, cutout, weights, raster_path)
    abw.plot_layout(shp, layout, dpi=50, title="_".join([c, weather_start, weather_end]))